In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

# We will use this port number for monitoring Spark Jobs
port=4050

spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Colab")\
        .config('spark.ui.port', str(port))\
        .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

spark

23/05/11 12:59:02 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.91.54.55 instead (on interface en0)
23/05/11 12:59:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/11 12:59:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# read /Users/husseinyounes/University/big_data/San-Francisco-Crime-Classification-bigdata/data/train.csv
import pandas as pd

file_path = '/Users/husseinyounes/University/big_data/San-Francisco-Crime-Classification-bigdata/data/train.csv'

# read the data
data = pd.read_csv(file_path)
data.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [4]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [47]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DateType, TimestampType

schema = StructType([
    StructField("Dates", TimestampType(), True),
    StructField("Category", StringType(), True),
    StructField("Descript", StringType(), True),
    StructField("DayOfWeek", StringType(), True),
    StructField("PdDistrict", StringType(), True),
    StructField("Resolution", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("X", StringType(), True),
    StructField("Y", StringType(), True)
])

# read the data
df = spark.read.csv(file_path, header=True, schema=schema)
df.show(5)

+-------------------+--------------+--------------------+---------+----------+--------------+--------------------+-------------------+------------------+
|              Dates|      Category|            Descript|DayOfWeek|PdDistrict|    Resolution|             Address|                  X|                 Y|
+-------------------+--------------+--------------------+---------+----------+--------------+--------------------+-------------------+------------------+
|2015-05-13 23:53:00|      WARRANTS|      WARRANT ARREST|Wednesday|  NORTHERN|ARREST, BOOKED|  OAK ST / LAGUNA ST|  -122.425891675136|  37.7745985956747|
|2015-05-13 23:53:00|OTHER OFFENSES|TRAFFIC VIOLATION...|Wednesday|  NORTHERN|ARREST, BOOKED|  OAK ST / LAGUNA ST|  -122.425891675136|  37.7745985956747|
|2015-05-13 23:33:00|OTHER OFFENSES|TRAFFIC VIOLATION...|Wednesday|  NORTHERN|ARREST, BOOKED|VANNESS AV / GREE...|   -122.42436302145|  37.8004143219856|
|2015-05-13 23:30:00| LARCENY/THEFT|GRAND THEFT FROM ...|Wednesday|  NORTHER

In [32]:
df.dtypes

[('Dates', 'timestamp'),
 ('Category', 'string'),
 ('Descript', 'string'),
 ('DayOfWeek', 'string'),
 ('PdDistrict', 'string'),
 ('Resolution', 'string'),
 ('Address', 'string'),
 ('X', 'string'),
 ('Y', 'string')]

In [33]:
df

Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425891675136,37.7745985956747
2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION...,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425891675136,37.7745985956747
2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION...,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREE...,-122.42436302145,37.8004143219856
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,NORTHERN,NONE,1500 Block of LOM...,-122.42699532676599,37.80087263276921
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,PARK,NONE,100 Block of BROD...,-122.438737622757,37.771541172057795
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,INGLESIDE,NONE,0 Block of TEDDY AV,-122.40325236121201,37.713430704116
2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE,AVALON AV / PERU AV,-122.423326976668,37.7251380403778
2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE,KIRKWOOD AV / DON...,-122.371274317441,37.7275640719518
2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,RICHMOND,NONE,600 Block of 47TH AV,-122.508194031117,37.776601260681204
2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,CENTRAL,NONE,JEFFERSON ST / LE...,-122.419087676747,37.8078015516515


In [63]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, HashingTF, IDF, Word2Vec, Tokenizer, MinMaxScaler, Word2VecModel, Word2Vec
from pyspark.sql.functions import udf, col, explode, dayofmonth, month, year, radians, sin, cos, lit
from pyspark.sql.types import DoubleType, StringType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCols
from pyspark.sql import DataFrame
import math
from pyspark.ml.linalg import SparseVector


cat_features = ['DayOfWeek', 'PdDistrict', 'Resolution', 'Category']
num_features = ['X', 'Y']
text_features = ['Descript']
date_features = ['Dates']

df_new = df

# convert the string labels to numbers
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in cat_features]

# convert the numbers to one hot vectors
encoders = [OneHotEncoder(inputCol=column+"_index", outputCol=column+"_vec") for column in cat_features]

# tokenize the text
tokenizers = [Tokenizer(inputCol=column, outputCol=f"{column}_words") for column in text_features]

# convert the text to vectors using Word2Vec
word2Vecs = [Word2Vec(vectorSize=3, minCount=0, inputCol=f"{column}_words", outputCol=f"{column}_vec") for column in text_features]

assembler = VectorAssembler(inputCols=['Descript_vec', 'DayOfWeek_vec', 'PdDistrict_vec', 'Resolution_vec'], outputCol="features")

# create a pipeline
pipeline = Pipeline(stages=indexers + encoders + tokenizers + word2Vecs + [assembler])

# fit the pipeline
model = pipeline.fit(df_new)
df_new = model.transform(df_new)

df_new

Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,DayOfWeek_index,PdDistrict_index,Resolution_index,Category_index,DayOfWeek_vec,PdDistrict_vec,Resolution_vec,Category_vec,Descript_words,Descript_vec,features
2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425891675136,37.7745985956747,1.0,2.0,1.0,7.0,"(6,[1],[1.0])","(9,[2],[1.0])","(16,[1],[1.0])","(38,[7],[1.0])","[warrant, arrest]",[-4.5020973682403...,"(34,[0,1,2,4,11,1..."
2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION...,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425891675136,37.7745985956747,1.0,2.0,1.0,1.0,"(6,[1],[1.0])","(9,[2],[1.0])","(16,[1],[1.0])","(38,[1],[1.0])","[traffic, violati...",[-3.6304053664207...,"(34,[0,1,2,4,11,1..."
2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION...,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREE...,-122.42436302145,37.8004143219856,1.0,2.0,1.0,1.0,"(6,[1],[1.0])","(9,[2],[1.0])","(16,[1],[1.0])","(38,[1],[1.0])","[traffic, violati...",[-3.6304053664207...,"(34,[0,1,2,4,11,1..."
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,NORTHERN,NONE,1500 Block of LOM...,-122.42699532676599,37.80087263276921,1.0,2.0,0.0,0.0,"(6,[1],[1.0])","(9,[2],[1.0])","(16,[0],[1.0])","(38,[0],[1.0])","[grand, theft, fr...",[0.37099074125289...,"(34,[0,1,2,4,11,1..."
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,PARK,NONE,100 Block of BROD...,-122.438737622757,37.771541172057795,1.0,8.0,0.0,0.0,"(6,[1],[1.0])","(9,[8],[1.0])","(16,[0],[1.0])","(38,[0],[1.0])","[grand, theft, fr...",[0.37099074125289...,"(34,[0,1,2,4,17,1..."
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,INGLESIDE,NONE,0 Block of TEDDY AV,-122.40325236121201,37.713430704116,1.0,6.0,0.0,0.0,"(6,[1],[1.0])","(9,[6],[1.0])","(16,[0],[1.0])","(38,[0],[1.0])","[grand, theft, fr...",[0.40975003838539...,"(34,[0,1,2,4,15,1..."
2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE,AVALON AV / PERU AV,-122.423326976668,37.7251380403778,1.0,6.0,0.0,5.0,"(6,[1],[1.0])","(9,[6],[1.0])","(16,[0],[1.0])","(38,[5],[1.0])","[stolen, automobile]",[-2.6523309051990...,"(34,[0,1,2,4,15,1..."
2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE,KIRKWOOD AV / DON...,-122.371274317441,37.7275640719518,1.0,3.0,0.0,5.0,"(6,[1],[1.0])","(9,[3],[1.0])","(16,[0],[1.0])","(38,[5],[1.0])","[stolen, automobile]",[-2.6523309051990...,"(34,[0,1,2,4,12,1..."
2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,RICHMOND,NONE,600 Block of 47TH AV,-122.508194031117,37.776601260681204,1.0,9.0,0.0,0.0,"(6,[1],[1.0])","(9,[],[])","(16,[0],[1.0])","(38,[0],[1.0])","[grand, theft, fr...",[0.37099074125289...,"(34,[0,1,2,4,18],..."
2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,CENTRAL,NONE,JEFFERSON ST / LE...,-122.419087676747,37.8078015516515,1.0,4.0,0.0,0.0,"(6,[1],[1.0])","(9,[4],[1.0])","(16,[0],[1.0])","(38,[0],[1.0])","[grand, theft, fr...",[0.37099074125289...,"(34,[0,1,2,4,13,1..."


In [65]:
# Category_index uniqu
df_new.select('Category_index').distinct().sort('Category_index').show()

+--------------+
|Category_index|
+--------------+
|           0.0|
|           1.0|
|           2.0|
|           3.0|
|           4.0|
|           5.0|
|           6.0|
|           7.0|
|           8.0|
|           9.0|
|          10.0|
|          11.0|
|          12.0|
|          13.0|
|          14.0|
|          15.0|
|          16.0|
|          17.0|
|          18.0|
|          19.0|
+--------------+
only showing top 20 rows



In [68]:
df_new

Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,DayOfWeek_index,PdDistrict_index,Resolution_index,Category_index,DayOfWeek_vec,PdDistrict_vec,Resolution_vec,Category_vec,Descript_words,Descript_vec,features
2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425891675136,37.7745985956747,1.0,2.0,1.0,7.0,"(6,[1],[1.0])","(9,[2],[1.0])","(16,[1],[1.0])","(38,[7],[1.0])","[warrant, arrest]",[-4.5020973682403...,"(34,[0,1,2,4,11,1..."
2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION...,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425891675136,37.7745985956747,1.0,2.0,1.0,1.0,"(6,[1],[1.0])","(9,[2],[1.0])","(16,[1],[1.0])","(38,[1],[1.0])","[traffic, violati...",[-3.6304053664207...,"(34,[0,1,2,4,11,1..."
2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION...,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREE...,-122.42436302145,37.8004143219856,1.0,2.0,1.0,1.0,"(6,[1],[1.0])","(9,[2],[1.0])","(16,[1],[1.0])","(38,[1],[1.0])","[traffic, violati...",[-3.6304053664207...,"(34,[0,1,2,4,11,1..."
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,NORTHERN,NONE,1500 Block of LOM...,-122.42699532676599,37.80087263276921,1.0,2.0,0.0,0.0,"(6,[1],[1.0])","(9,[2],[1.0])","(16,[0],[1.0])","(38,[0],[1.0])","[grand, theft, fr...",[0.37099074125289...,"(34,[0,1,2,4,11,1..."
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,PARK,NONE,100 Block of BROD...,-122.438737622757,37.771541172057795,1.0,8.0,0.0,0.0,"(6,[1],[1.0])","(9,[8],[1.0])","(16,[0],[1.0])","(38,[0],[1.0])","[grand, theft, fr...",[0.37099074125289...,"(34,[0,1,2,4,17,1..."
2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,INGLESIDE,NONE,0 Block of TEDDY AV,-122.40325236121201,37.713430704116,1.0,6.0,0.0,0.0,"(6,[1],[1.0])","(9,[6],[1.0])","(16,[0],[1.0])","(38,[0],[1.0])","[grand, theft, fr...",[0.40975003838539...,"(34,[0,1,2,4,15,1..."
2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE,AVALON AV / PERU AV,-122.423326976668,37.7251380403778,1.0,6.0,0.0,5.0,"(6,[1],[1.0])","(9,[6],[1.0])","(16,[0],[1.0])","(38,[5],[1.0])","[stolen, automobile]",[-2.6523309051990...,"(34,[0,1,2,4,15,1..."
2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE,KIRKWOOD AV / DON...,-122.371274317441,37.7275640719518,1.0,3.0,0.0,5.0,"(6,[1],[1.0])","(9,[3],[1.0])","(16,[0],[1.0])","(38,[5],[1.0])","[stolen, automobile]",[-2.6523309051990...,"(34,[0,1,2,4,12,1..."
2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,RICHMOND,NONE,600 Block of 47TH AV,-122.508194031117,37.776601260681204,1.0,9.0,0.0,0.0,"(6,[1],[1.0])","(9,[],[])","(16,[0],[1.0])","(38,[0],[1.0])","[grand, theft, fr...",[0.37099074125289...,"(34,[0,1,2,4,18],..."
2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM ...,Wednesday,CENTRAL,NONE,JEFFERSON ST / LE...,-122.419087676747,37.8078015516515,1.0,4.0,0.0,0.0,"(6,[1],[1.0])","(9,[4],[1.0])","(16,[0],[1.0])","(38,[0],[1.0])","[grand, theft, fr...",[0.37099074125289...,"(34,[0,1,2,4,13,1..."


In [66]:
# classify using Random Forest
from pyspark.ml.classification import RandomForestClassifier

# split the data into train and test
train_data, test_data = df_new.randomSplit([0.7, 0.3], seed=42)

# create the model
rf = RandomForestClassifier(labelCol="Category_index", featuresCol="features", numTrees=10)

# fit the model
rf_model = rf.fit(train_data)

In [69]:
predictions = rf_model.transform(test_data)

In [71]:
predictions.select(['Descript', 'Category', 'Category_index', 'prediction']).show(5)

+--------------------+-------------+--------------+----------+
|            Descript|     Category|Category_index|prediction|
+--------------------+-------------+--------------+----------+
|   STOLEN AUTOMOBILE|VEHICLE THEFT|           5.0|       5.0|
|AIDED CASE, MENTA...| NON-CRIMINAL|           2.0|       2.0|
|MALICIOUS MISCHIE...|    VANDALISM|           6.0|       6.0|
|FRAUDULENT USE OF...|        FRAUD|          12.0|       3.0|
|GRAND THEFT FROM ...|LARCENY/THEFT|           0.0|       0.0|
+--------------------+-------------+--------------+----------+
only showing top 5 rows



In [74]:
# get the prediction accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="Category_index", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)

Accuracy = 0.699542


In [73]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="Category_index", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"RMSE: {rmse}")

RMSE: 5.688265282441608


In [75]:
# evaluate using F1 score
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="Category_index", predictionCol="prediction", metricName="f1")
f1_score = evaluator.evaluate(predictions)
print(f"F1 score: {f1_score}")

F1 score: 0.6453957626853973
